In [3]:
pip install opencv-python numpy pyautogui pyaudio wave


  Using cached PyAutoGUI-0.9.54.tar.gz (61 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached PyMsgBox-1.0.9.tar.gz (18 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pytweening-1.2.0.tar.gz (171 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'd

In [4]:
pip list

Package                   VersionNote: you may need to restart the kernel to use updated packages.

------------------------- --------------
anyio                     4.8.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     25.1.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.2.0
certifi                   2024.12.14
cffi                      1.17.1
charset-normalizer        3.4.1
colorama                  0.4.6
comm                      0.2.2
debugpy                   1.8.12
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.2.0
fastjsonschema            2.21.1
fqdn                      1.5.1
h11                       0.14.0
httpcore                  1.0.7
httpx                     0.28.1
idna                      3.10
ipykernel                 6.29.5
ipython     

In [7]:
import cv2
import numpy as np
import pyautogui
import pyaudio
import wave
import threading

In [10]:
# 📌 녹화 설정
SCREEN_SIZE = (1920, 1080)  # 화면 해상도 (여기에 자신의 해상도를 설정하세요)
FPS = 20.0  # 초당 프레임 수
VIDEO_FILENAME = "recorded_video.mp4"  # 저장될 영상 파일명
AUDIO_FILENAME = "recorded_audio.wav"  # 저장될 오디오 파일명
OUTPUT_FILENAME = "final_output.mp4"  # 최종 합친 파일

# 🎥 OpenCV 비디오 코덱 설정
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter(VIDEO_FILENAME, fourcc, FPS, SCREEN_SIZE)

# 🎤 오디오 녹음 설정
FORMAT = pyaudio.paInt16
CHANNELS = 2  # 스테레오 녹음
RATE = 44100  # 샘플링 레이트
CHUNK = 1024  # 버퍼 크기

audio = pyaudio.PyAudio()
stream = audio.open(format=FORMAT, channels=1, rate=RATE, input=True, frames_per_buffer=CHUNK)
frames = []


def record_screen():
    """화면을 녹화하는 함수"""
    while recording:
        img = pyautogui.screenshot()  # 화면 캡처
        frame = np.array(img)  # NumPy 배열 변환
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # OpenCV 색상 변환
        video_writer.write(frame)  # 프레임 저장


def record_audio():
    """오디오를 녹음하는 함수"""
    while recording:
        data = stream.read(CHUNK)
        frames.append(data)


# 🎬 녹화 시작
recording = True
screen_thread = threading.Thread(target=record_screen)
audio_thread = threading.Thread(target=record_audio)

screen_thread.start()
audio_thread.start()

input("🎥 녹화를 시작했습니다. 중지하려면 Enter를 누르세요...\n")

# 🛑 녹화 중지
recording = False
screen_thread.join()
audio_thread.join()

# 🎥 비디오 파일 저장 완료
video_writer.release()
cv2.destroyAllWindows()

# 🎤 오디오 파일 저장 완료
stream.stop_stream()
stream.close()
audio.terminate()

wf = wave.open(AUDIO_FILENAME, "wb")
wf.setnchannels(CHANNELS)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b"".join(frames))
wf.close()

print("✅ 화면과 오디오 녹화가 완료되었습니다.")

# 🎬 비디오와 오디오 합치기 (FFmpeg 필요)
import os

merge_command = f'ffmpeg -i {VIDEO_FILENAME} -i {AUDIO_FILENAME} -c:v copy -c:a aac -strict experimental {OUTPUT_FILENAME}'
os.system(merge_command)

print(f"✅ 최종 파일 저장 완료: {OUTPUT_FILENAME}")

Exception in thread Thread-7 (record_screen):
Traceback (most recent call last):
  File "C:\anaconda3\envs\videoenv\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\anaconda3\envs\videoenv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\anaconda3\envs\videoenv\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\손민구\AppData\Local\Temp\ipykernel_24512\3583464412.py", line 26, in record_screen
    img = pyautogui.screenshot()  # 화면 캡처
  File "C:\anaconda3\envs\videoenv\Lib\site-packages\pyautogui\__init__.py", line 228, in _couldNotImportPyScreeze
    raise PyAutoGUIException(
        "PyAutoGUI was unable to import pyscreeze. (This is likely because you're running a version of Python that Pillow (which pyscreeze depends on) doesn't support currently.) Please inst

🎥 녹화를 시작했습니다. 중지하려면 Enter를 누르세요...
 


✅ 화면과 오디오 녹화가 완료되었습니다.
✅ 최종 파일 저장 완료: final_output.mp4
